### MLP with Tensorboard

see:  https://github.com/nlintz/TensorFlow-Tutorials/blob/master/9_tensorboard.py


In [1]:
# A Multilayer Perceptron implementation example using TensorFlow library.
# This example is using the MNIST database of handwritten digits
# (http://yann.lecun.com/exdb/mnist/)=


# Author: Aymeric Damien
# Project: https://github.com/aymericdamien/TensorFlow-Examples/

In [2]:
import tensorflow as tf
import numpy as np
import input_data

In [3]:
# Parameters
#learning_rate = 0.001
training_epochs = 100
batch_size = 100
display_step = 1
total_batch = 55000/100

In [4]:
# Network Parameters
n_hidden_1 = 1000 # 1st layer num features
n_hidden_2 = 500 # 2nd layer num features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

std_0 = 1.0/np.sqrt(n_input)
std_h1 = 1.0/np.sqrt(n_hidden_1)
std_h2 = 1.0/np.sqrt(n_hidden_2)


In [5]:
# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [6]:
# dropout parameters
pv = tf.placeholder("float")
ph = tf.placeholder("float")

In [7]:
def multilayer_perceptron(_X, _weights, _biases, _pv, _ph):   
    # Add layer name scopes for better graph visualization
    with tf.name_scope("layer1"):
        _Xd = tf.nn.dropout(_X, _pv)
        layer_1 = tf.nn.relu(tf.matmul(_Xd, _weights['h1']))
    with tf.name_scope("layer2"):
        layer_1d = tf.nn.dropout(layer_1, _ph)
        layer_2 = tf.nn.relu(tf.matmul(layer_1d, _weights['h2']))
    with tf.name_scope("layer3"):
        layer_3 = tf.nn.dropout(layer_2, _ph)
        return tf.matmul(layer_3, _weights['out'])

In [8]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=std_0)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=std_h1)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=std_h2))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1], stddev=std_h1)),
    'b2': tf.Variable(tf.random_normal([n_hidden_2], stddev=std_h2)),
    'out': tf.Variable(tf.random_normal([n_classes], stddev=0.3))
}

In [9]:
# Construct model
pred = multilayer_perceptron(x, weights, biases, pv, ph )

In [10]:
# Add histogram summaries for weights
tf.histogram_summary("w1_summ", weights['h1'])
tf.histogram_summary("w2_summ", weights['h2'])
tf.histogram_summary("wo_summ", weights['out'])

<tf.Tensor 'HistogramSummary_2:0' shape=() dtype=string>

In [11]:
with tf.name_scope("cost"):
    cost =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) 
    optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
    # Add scalar summary for cost
    tf.scalar_summary("cost", cost)


In [12]:
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1)) # Count correct predictions
    # Calculate accuracy
    acc_op = tf.reduce_mean(tf.cast(correct_prediction, "float"))  # Cast boolean to float to average
    # Add scalar summary for accuracy
    tf.scalar_summary("accuracy", acc_op)

In [13]:
mnist = input_data.read_data_sets(".", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

/Users/charlesmartin14/anaconda2/lib/python2.7/gzip.py:275: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  chunk = self.extrabuf[offset: offset + size]
input_data.py:35: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


In [14]:
print trX.shape, trY.shape
print teX.shape, teY.shape

(55000, 784) (55000, 10)
(10000, 784) (10000, 10)


In [15]:
55000./125.

440.0

In [16]:
!rm -rf ./logs/nn_logs

In [17]:
with tf.Session() as sess:
    # create a log writer. run 'tensorboard --logdir=./logs/nn_logs'
    writer = tf.train.SummaryWriter("./logs/nn_logs", sess.graph) # for 0.8
    merged = tf.merge_all_summaries()

    # you need to initialize all variables
    tf.initialize_all_variables().run()

    for i in range(5000):
        for start, end in zip(range(0, len(trX), 125), range(125, len(trX), 125)):
            #print start, end
            sess.run(optimizer, feed_dict={x: trX[start:end], y: trY[start:end], pv: 0.8, ph: 0.5})
        summary , acc = sess.run([merged, acc_op], feed_dict={x: teX, y: teY, pv: 1.0, ph: 1.0})
        writer.add_summary(summary, i)  # Write summary
        print(i, acc)                   # Report the accuracy
        
    writer.flush()

(0, 0.94849998)
(1, 0.9666)
(2, 0.97219998)
(3, 0.97460002)
(4, 0.97750002)
(5, 0.97820002)
(6, 0.97979999)
(7, 0.97930002)
(8, 0.98049998)
(9, 0.98119998)
(10, 0.98180002)
(11, 0.9795)
(12, 0.98030001)
(13, 0.97899997)
(14, 0.98159999)
(15, 0.98140001)
(16, 0.98100001)
(17, 0.9817)
(18, 0.9824)
(19, 0.9817)
(20, 0.98199999)
(21, 0.9849)
(22, 0.98400003)
(23, 0.98259997)
(24, 0.98460001)
(25, 0.98189998)
(26, 0.98320001)
(27, 0.98400003)
(28, 0.98400003)
(29, 0.9842)
(30, 0.98119998)
(31, 0.98269999)
(32, 0.9824)
(33, 0.98409998)
(34, 0.98189998)
(35, 0.98299998)
(36, 0.98269999)
(37, 0.98379999)
(38, 0.98220003)
(39, 0.98400003)
(40, 0.9842)
(41, 0.9831)
(42, 0.98339999)
(43, 0.98409998)
(44, 0.98320001)
(45, 0.98100001)
(46, 0.98250002)
(47, 0.98460001)
(48, 0.98369998)
(49, 0.9817)
(50, 0.98299998)
(51, 0.98290002)
(52, 0.98320001)
(53, 0.98269999)
(54, 0.98299998)
(55, 0.98250002)
(56, 0.98400003)
(57, 0.98329997)
(58, 0.98229998)
(59, 0.98460001)
(60, 0.98360002)
(61, 0.98439997)
